<a href="https://colab.research.google.com/github/Tanush2710/AI-Driven-cybersecurity-threat-prediction-platform/blob/main/SQL_Task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Setup (imports)

In [ ]:
# Cell 1: Setup imports for SQL demo
import sqlite3
import pandas as pd
from io import StringIO


 Create in-memory DB and tables

In [ ]:
con = sqlite3.connect(":memory:")
cur = con.cursor()

cur.executescript("""
CREATE TABLE users (
    user_id INTEGER PRIMARY KEY,
    name TEXT NOT NULL,
    age INTEGER,
    city TEXT
);

CREATE TABLE orders (
    order_id INTEGER PRIMARY KEY,
    user_id INTEGER,
    product TEXT,
    amount REAL,
    order_date TEXT,
    FOREIGN KEY(user_id) REFERENCES users(user_id)
);
""")
con.commit()
print("DB and tables created.")

DB and tables created.


Insert data

In [ ]:
users = [(1,'Aisha',28,'Gwalior'),
         (2,'Rahul',22,'Bhopal'),
         (3,'Meera',31,'Indore'),
         (4,'Arjun',24,'Gwalior'),
         (5,'Sneha',29,'Bhopal')]

orders = [(101,1,'Laptop',55000.0,'2025-02-10'),
          (102,2,'Mouse',499.0,'2025-03-05'),
          (103,1,'Keyboard',1200.0,'2025-03-12'),
          (104,3,'Monitor',8200.0,'2025-04-01'),
          (105,4,'Tablet',15000.0,'2025-04-15'),
          (106,2,'Headphones',1500.0,'2025-05-05'),
          (107,5,'Keyboard',1300.0,'2025-05-20')]

cur.executemany("INSERT INTO users VALUES (?,?,?,?)", users)
cur.executemany("INSERT INTO orders VALUES (?,?,?,?,?)", orders)
con.commit()
print("Inserted users:", len(users), "Inserted orders:", len(orders))

Inserted users: 5 Inserted orders: 7


Basic SELECT, WHERE, ORDER BY

In [ ]:
q1 = "SELECT * FROM users;"
print("All users:")
for row in cur.execute(q1):
    print(row)

q2 = "SELECT name,city FROM users WHERE age >= 25 ORDER BY age DESC;"
print("\nUsers age >= 25 (ordered):")
for row in cur.execute(q2):
    print(row)

All users:
(1, 'Aisha', 28, 'Gwalior')
(2, 'Rahul', 22, 'Bhopal')
(3, 'Meera', 31, 'Indore')
(4, 'Arjun', 24, 'Gwalior')
(5, 'Sneha', 29, 'Bhopal')

Users age >= 25 (ordered):
('Meera', 'Indore')
('Sneha', 'Bhopal')
('Aisha', 'Gwalior')


INNER JOIN (users with their orders)

In [ ]:
q3 = """
SELECT u.user_id, u.name, o.order_id, o.product, o.amount
FROM users u
JOIN orders o ON u.user_id = o.user_id
ORDER BY u.user_id, o.order_id;
"""
print("INNER JOIN results:")
for row in cur.execute(q3):
    print(row)

INNER JOIN results:
(1, 'Aisha', 101, 'Laptop', 55000.0)
(1, 'Aisha', 103, 'Keyboard', 1200.0)
(2, 'Rahul', 102, 'Mouse', 499.0)
(2, 'Rahul', 106, 'Headphones', 1500.0)
(3, 'Meera', 104, 'Monitor', 8200.0)
(4, 'Arjun', 105, 'Tablet', 15000.0)
(5, 'Sneha', 107, 'Keyboard', 1300.0)


LEFT JOIN + GROUP BY aggregation

In [ ]:
q4 = """
SELECT u.user_id, u.name, IFNULL(SUM(o.amount),0) as total_spent, COUNT(o.order_id) as orders_count
FROM users u
LEFT JOIN orders o ON u.user_id = o.user_id
GROUP BY u.user_id, u.name
ORDER BY total_spent DESC;
"""
print("Total spent per user (LEFT JOIN + GROUP BY):")
for row in cur.execute(q4):
    print(row)

Total spent per user (LEFT JOIN + GROUP BY):
(1, 'Aisha', 56200.0, 2)
(4, 'Arjun', 15000.0, 1)
(3, 'Meera', 8200.0, 1)
(2, 'Rahul', 1999.0, 2)
(5, 'Sneha', 1300.0, 1)


Transactions (rollback demo)

In [ ]:
cur.execute("BEGIN;")
cur.execute("INSERT INTO users VALUES (6,'Vikram',35,'Gwalior');")
cur.execute("INSERT INTO orders VALUES (108,6,'Camera',25000.0,'2025-06-01');")
cur.execute("ROLLBACK;")
print("Rolled back transaction. Verify user_id 6 exists? ->",
      list(cur.execute("SELECT * FROM users WHERE user_id=6;")))

Rolled back transaction. Verify user_id 6 exists? -> []


Create index and EXPLAIN QUERY PLAN

In [ ]:
cur.execute("CREATE INDEX idx_orders_user ON orders(user_id);")
explain = list(cur.execute("EXPLAIN QUERY PLAN " + q3))
print("EXPLAIN QUERY PLAN output:")
for row in explain:
    print(row)

EXPLAIN QUERY PLAN output:
(5, 0, 0, 'SCAN u')
(7, 0, 0, 'SEARCH o USING INDEX idx_orders_user (user_id=?)')


Export SQL result to pandas

In [ ]:
df_orders = pd.read_sql_query("SELECT o.order_id, u.name, o.product, o.amount FROM orders o JOIN users u ON o.user_id=u.user_id;", con)
print(df_orders.head())

   order_id   name   product   amount
0       101  Aisha    Laptop  55000.0
1       102  Rahul     Mouse    499.0
2       103  Aisha  Keyboard   1200.0
3       104  Meera   Monitor   8200.0
4       105  Arjun    Tablet  15000.0


Clean up

In [ ]:
con.close()
print("Closed DB.")

Closed DB.
